In [17]:
%load_ext autoreload
%autoreload 2

import torch
from utility.attentionAutoEncoder import AttentionAutoEncoder, concate_data, AttentionDataset, get_return_from_batch
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model = AttentionAutoEncoder(3, 5, 2)

In [3]:
input_data = torch.randn(3, 5)
input_data

tensor([[-0.5975, -0.6707, -0.1882, -0.2758,  0.0063],
        [-0.2311,  0.8977, -0.6596, -0.2374, -0.7762],
        [ 0.1455,  0.4327,  0.6082,  1.6839,  0.6171]])

In [4]:
model(input_data)

tensor([[ 0.4820,  6.7751, -6.6022,  4.5491,  7.7067]], grad_fn=<MmBackward0>)

In [5]:
import pandas as pd
from utility.data_processing import set_time_data

In [6]:
dfs = dict()
for ticker in ['btc', 'eth', 'doge', 'xrp', 'ada']:
  currency_metrics = pd.read_csv(f'../data/{ticker}_metrics_5_years.csv')
  set_currency_metrics = set_time_data(currency_metrics)
  dfs[ticker] = set_currency_metrics

In [7]:
res = concate_data(dfs)

In [8]:
X_t = res[:, :, 0]

In [9]:
model(X_t)

tensor([[-0.0183, -0.0017, -0.0749,  0.0514,  0.0709]], grad_fn=<MmBackward0>)

In [10]:
X_t.shape[-1]

5

In [ ]:
# Create DataSet
dataset = AttentionDataset(res)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=50, shuffle=True, num_workers=0)

# Iterate through DataLoader
for batch in dataloader:
    print(batch.shape)  # Output: torch.Size([10, 5])
    break  # To print only one batch

torch.Size([50, 3, 5])


In [33]:
# get_return_from_batch(batch)

In [14]:
# model(batch)

In [15]:
import torch.nn as nn
import torch.optim as optim

In [16]:
criterion = nn.MSELoss()  # Mean Squared Error for reconstruction
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
num_epochs = 50

for epoch in range(num_epochs):
    total_loss = 0
    for X_t in dataloader:
        R_T = get_return_from_batch(X_t)
        # Forward pass
        R_T_hat = model(X_t)
        loss = criterion(R_T_hat, R_T)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/50], Loss: 0.2295
Epoch [2/50], Loss: 0.1994
Epoch [3/50], Loss: 0.1787
Epoch [4/50], Loss: 0.1572
Epoch [5/50], Loss: 0.1416
Epoch [6/50], Loss: 0.1265
Epoch [7/50], Loss: 0.1166
Epoch [8/50], Loss: 0.1029
Epoch [9/50], Loss: 0.0930
Epoch [10/50], Loss: 0.0844
Epoch [11/50], Loss: 0.0751
Epoch [12/50], Loss: 0.0674
Epoch [13/50], Loss: 0.0613
Epoch [14/50], Loss: 0.0545
Epoch [15/50], Loss: 0.0486
Epoch [16/50], Loss: 0.0437
Epoch [17/50], Loss: 0.0391
Epoch [18/50], Loss: 0.0348
Epoch [19/50], Loss: 0.0316
Epoch [20/50], Loss: 0.0282
Epoch [21/50], Loss: 0.0255
Epoch [22/50], Loss: 0.0232
Epoch [23/50], Loss: 0.0209
Epoch [24/50], Loss: 0.0190
Epoch [25/50], Loss: 0.0177
Epoch [26/50], Loss: 0.0163
Epoch [27/50], Loss: 0.0149
Epoch [28/50], Loss: 0.0139
Epoch [29/50], Loss: 0.0128
Epoch [30/50], Loss: 0.0120
Epoch [31/50], Loss: 0.0113
Epoch [32/50], Loss: 0.0106
Epoch [33/50], Loss: 0.0099
Epoch [34/50], Loss: 0.0093
Epoch [35/50], Loss: 0.0090
Epoch [36/50], Loss: 0.0084
E